## Plot GPS Coordinates to Map 
- varify coastal ports
- https://towardsdatascience.com/easy-steps-to-plot-geographic-data-on-a-map-python-11217859a2db
- https://www.openstreetmap.org/export#map=2/23.9/26.0
- https://www.kaggle.com/code/subinium/how-to-use-folium-geospatial-data/notebook
- https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [37]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [38]:
#load datasets
coastal = pd.read_csv('CoastalPorts.csv')
other = pd.read_csv('OtherPorts.csv')

In [39]:
coastal.head()

,Country Code,Port Location,Port Name,Port Authority,Address,Phone,Fax,800 Number,Email,Web Site,Latitude,Longitude,UN/LOCODE,Port Type,Port Size,Country
0,ALB,Durres,Port of Durres,Durres Port Authority,Kapitenerija e Portit\nL Nr 1 Rruga Tregtare\n...,355 52 23115,355 52 22028,NaN,apd@san.com.al,NaN,"41° 18' 28"" N","19° 27' 17"" E",ALDRZ,Seaport,Small,NaN
1,ALB,Sarande,Port of Sarande,Sarande Port Authority,Port Office\nSarande Port\nSarande\nAlbania,355 85 25827,355 85 25827,NaN,NaN,NaN,"39° 52' 19"" N","20° 0' 20"" E",ALSAR,"Pier, Jetty or Wharf",Small,NaN
2,ALB,Vlore,Port of Vlore,Vlore Drejtoria e Portit Detar,Albania,NaN,NaN,NaN,porti-vlore@aul.com.al,NaN,"40° 28' 7"" N","19° 27' 36"" E",ALVOA,Seaport,Small,NaN
3,DZA,Algiers,Port of Algiers,Entreprise Portuaire d'Alger,"02 Rue d'Angkor\nBP 259\nAlgiers, Gare 259\nAl...",213 21 423614,213 21 423603,NaN,epal@portalger.com.dz,www.portalger.com.dz,"36° 46' 25"" N","3° 4' 2"" E",DZALG,Deepwater Seaport,Large,NaN
4,DZA,Annaba,Port of Annaba,Entreprise Portuaire de Annaba,Mole Cigogne\nQuai Nord\nAnnaba 1232 - 23000\n...,213 38 86 3143,213 38 86 54 15,NaN,epan@annaba-port.com,www.annaba-port.com,"36° 54' 2"" N","7° 46' 21"" E",DZAAE,Seaport,Medium,NaN


In [40]:
len(coastal)

2479

In [41]:
#coastal = coastal.drop_duplicates(keep='last').reset_index()

In [42]:
#len(coastal)

In [43]:
#subset data into country code, port name, lat, long, etc
coastal_subset = coastal.drop(['Port Authority', 'Address', 'Phone', 'Fax', '800 Number', 'Email', 'Web Site', 'Country'], axis=1)
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size
0,ALB,Durres,Port of Durres,"41° 18' 28"" N","19° 27' 17"" E",ALDRZ,Seaport,Small
1,ALB,Sarande,Port of Sarande,"39° 52' 19"" N","20° 0' 20"" E",ALSAR,"Pier, Jetty or Wharf",Small
2,ALB,Vlore,Port of Vlore,"40° 28' 7"" N","19° 27' 36"" E",ALVOA,Seaport,Small
3,DZA,Algiers,Port of Algiers,"36° 46' 25"" N","3° 4' 2"" E",DZALG,Deepwater Seaport,Large
4,DZA,Annaba,Port of Annaba,"36° 54' 2"" N","7° 46' 21"" E",DZAAE,Seaport,Medium


In [44]:
#convert degree minute second to degree decimal for latitudes
lat_list = []
for latitude in coastal_subset['Latitude']:
    #print(latitude)
    lat = latitude
    deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
    lat = round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)
    #print(lat)
    lat_list.append(lat)
coastal_subset['Latitude'] = lat_list

In [45]:
#convert degree minute second to degree decimal for latitudes
lon_list = []
for longitude in coastal_subset['Longitude']:
    #print(longitude)
    lon = longitude
    deg, minutes, seconds, direction =  re.split('[°\'"]', lon)
    lon = round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)
    #print(lon)
    lon_list.append(lon)
coastal_subset['Longitude'] = lon_list

In [46]:
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size
0,ALB,Durres,Port of Durres,41.307778,19.454722,ALDRZ,Seaport,Small
1,ALB,Sarande,Port of Sarande,39.871944,20.005556,ALSAR,"Pier, Jetty or Wharf",Small
2,ALB,Vlore,Port of Vlore,40.468611,19.460000,ALVOA,Seaport,Small
3,DZA,Algiers,Port of Algiers,36.773611,3.067222,DZALG,Deepwater Seaport,Large
4,DZA,Annaba,Port of Annaba,36.900556,7.772500,DZAAE,Seaport,Medium


In [47]:
#create new column of combine Latitude and Longitude as list
coastal_subset['lat_long'] = coastal_subset[['Latitude', 'Longitude']].to_numpy().tolist()

In [48]:
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size,lat_long
0,ALB,Durres,Port of Durres,41.307778,19.454722,ALDRZ,Seaport,Small,"[41.307778, 19.454722]"
1,ALB,Sarande,Port of Sarande,39.871944,20.005556,ALSAR,"Pier, Jetty or Wharf",Small,"[39.871944, 20.005556]"
2,ALB,Vlore,Port of Vlore,40.468611,19.460000,ALVOA,Seaport,Small,"[40.468611, 19.46]"
3,DZA,Algiers,Port of Algiers,36.773611,3.067222,DZALG,Deepwater Seaport,Large,"[36.773611, 3.067222]"
4,DZA,Annaba,Port of Annaba,36.900556,7.772500,DZAAE,Seaport,Medium,"[36.900556, 7.7725]"


In [49]:
#new column with combined location description
#ex: Port of Durres, Seaport - Small
coastal_subset['details'] = coastal_subset['Port Name'] + ', ' + coastal_subset['Port Type'] + ' - ' + coastal_subset['Port Size']
coastal_subset.head()

,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size,lat_long,details
0,ALB,Durres,Port of Durres,41.307778,19.454722,ALDRZ,Seaport,Small,"[41.307778, 19.454722]","Port of Durres, Seaport - Small"
1,ALB,Sarande,Port of Sarande,39.871944,20.005556,ALSAR,"Pier, Jetty or Wharf",Small,"[39.871944, 20.005556]","Port of Sarande, Pier, Jetty or Wharf - Small"
2,ALB,Vlore,Port of Vlore,40.468611,19.460000,ALVOA,Seaport,Small,"[40.468611, 19.46]","Port of Vlore, Seaport - Small"
3,DZA,Algiers,Port of Algiers,36.773611,3.067222,DZALG,Deepwater Seaport,Large,"[36.773611, 3.067222]","Port of Algiers, Deepwater Seaport - Large"
4,DZA,Annaba,Port of Annaba,36.900556,7.772500,DZAAE,Seaport,Medium,"[36.900556, 7.7725]","Port of Annaba, Seaport - Medium"


In [50]:
#save new table
coastal_subset.to_csv("CoastalPortsSubsetData.csv", index=False)

## Build map
Resources: 
- https://www.earthdatascience.org/courses/scientists-guide-to-plotting-data-in-python/plot-spatial-data/customize-raster-plots/interactive-maps/
- https://python-visualization.github.io/folium/

In [51]:
# Import necessary packages
import os 
import folium
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es

### Map of all Coastal Ports

In [52]:
locations = coastal_subset['lat_long']
len(locations)
locations[1]

[39.871944, 20.005556]

In [53]:
#map of all coastal ports
portmap = folium.Map(tiles='Stamen Terrain', max_bounds=True)

marker_cluster = plugins.MarkerCluster().add_to(portmap)
for point in range(0, len(locations)):
    folium.Marker(locations[point],
                  popup=coastal_subset['details'][point],
                  tooltip=coastal_subset['details'][point]
                 ).add_to(marker_cluster)
portmap

In [54]:
# Export folium map as HTML 
portmap.save("CoastalPortMap.html")

### Map of Largest Coastal Ports

In [57]:
largest_coastal = coastal_subset.loc[coastal_subset['Port Size'].isin(['Large', 'Very Large'])].reset_index()
largest_coastal.head()

,index,Country Code,Port Location,Port Name,Latitude,Longitude,UN/LOCODE,Port Type,Port Size,lat_long,details
0,3,DZA,Algiers,Port of Algiers,36.773611,3.067222,DZALG,Deepwater Seaport,Large,"[36.773611, 3.067222]","Port of Algiers, Deepwater Seaport - Large"
1,7,DZA,Bejaia,Port of Bejaia,36.750000,5.090000,DZBJA,Deepwater Seaport,Large,"[36.75, 5.09]","Port of Bejaia, Deepwater Seaport - Large"
2,30,ARG,Buenos Aires,Port of Buenos Aires,-34.599167,-58.371389,ARBUE,Deepwater Seaport,Large,"[-34.599167, -58.371389]","Port of Buenos Aires, Deepwater Seaport - Large"
3,36,ARG,Dock Sud,Port of Dock Sud,-34.634167,-58.348056,NaN,Deepwater Seaport,Large,"[-34.634167, -58.348056]","Port of Dock Sud, Deepwater Seaport - Large"
4,65,AUS,Brisbane,Port of Brisbane,-27.382500,153.169444,AUBNE,Seaport,Large,"[-27.3825, 153.169444]","Port of Brisbane, Seaport - Large"


In [60]:
largest_coastal.to_csv("LargestCoastalPortsSubset.csv", index=False)

In [58]:
largest_locations = largest_coastal['lat_long']
len(largest_locations)
largest_locations[1]

[36.75, 5.09]

In [61]:
#map of largest coastal ports
large_ports_map = folium.Map(tiles='Stamen Terrain', max_bounds=True)

large_marker_cluster = plugins.MarkerCluster().add_to(large_ports_map)
#draw = plugins.Draw(export=True).add_to(large_ports_map) #allows us to draw on it and export it
plugins.Fullscreen(position='topright', # ‘topleft’, default=‘topright’, ‘bottomleft’, ‘bottomright’ 
                   title='FULL SCREEN ON', 
                   title_cancel='FULL SCREEN OFF',
                   force_separate_button=True
                  ).add_to(large_ports_map)

for point in range(0, len(largest_locations)):
    folium.Marker(largest_locations[point],
                  popup=largest_coastal['details'][point],
                  tooltip=largest_coastal['details'][point],
                 ).add_to(large_marker_cluster)
large_ports_map

In [62]:
# Export folium map as HTML 
large_ports_map.save("LargePortsMap.html")

## Future Ideas 
- add heatmap of whale locations/movement

_______________________________

In [20]:
## Example
#map Durres
'''
durres = folium.Map(location=[41.307778, 19.454722],
                    tiles='Stamen Terrain') #zoom_start= 15 (default = 10)
#add marker for durres
folium.Marker(
    location =[41.307778, 19.454722],
    popup='Port of Durres, Seaport - Small', #pop-up label for marker
    tooltip='Port of Durres, Seaport - Small' #hover over label for marker
).add_to(durres)

#durres

#can draw on map and export it
world = folium.Map() # full map
draw = plugins.Draw(export=True)
draw.add_to(world)
#world
'''

"\ndurres = folium.Map(location=[41.307778, 19.454722],\n                    tiles='Stamen Terrain') #zoom_start= 15 (default = 10)\n#add marker for durres\nfolium.Marker(\n    location =[41.307778, 19.454722],\n    popup='Port of Durres, Seaport - Small', #pop-up label for marker\n    tooltip='Port of Durres, Seaport - Small' #hover over label for marker\n).add_to(durres)\n\n#durres\n\n#can draw on map and export it\nworld = folium.Map() # full map\ndraw = plugins.Draw(export=True)\ndraw.add_to(world)\n#world\n"

In [21]:
## Test 
'''
coords = coastal_subset['lat_long'][0:3]
details = coastal_subset['details'][0:3]
for coord in coords:
    world = folium.Map(location=coord,
                    tiles='Stamen Terrain') # full map
    folium.Marker(
    location = coord,
    popup=details, #pop-up label for marker
    tooltip=details #hover over label for marker
    ).add_to(world)
world '''

"\ncoords = coastal_subset['lat_long'][0:3]\ndetails = coastal_subset['details'][0:3]\nfor coord in coords:\n    world = folium.Map(location=coord,\n                    tiles='Stamen Terrain') # full map\n    folium.Marker(\n    location = coord,\n    popup=details, #pop-up label for marker\n    tooltip=details #hover over label for marker\n    ).add_to(world)\nworld "

In [22]:
#Example
'''
map2 = folium.Map(location=[38.9, -77.05], tiles='Stamen Terrain', zoom_start=11)
marker_cluster = folium.MarkerCluster().add_to(map2)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_counters['Name'][point], 
                  icon=folium.Icon(color='darkblue', 
                                   icon_color='white', 
                                   icon='male', 
                                   angle=0, 
                                   prefix='fa')).add_to(marker_cluster)
map2
'''

"\nmap2 = folium.Map(location=[38.9, -77.05], tiles='Stamen Terrain', zoom_start=11)\nmarker_cluster = folium.MarkerCluster().add_to(map2)\nfor point in range(0, len(locationlist)):\n    folium.Marker(locationlist[point], popup=df_counters['Name'][point], \n                  icon=folium.Icon(color='darkblue', \n                                   icon_color='white', \n                                   icon='male', \n                                   angle=0, \n                                   prefix='fa')).add_to(marker_cluster)\nmap2\n"

In [23]:
## Test
'''
#change coordinates to decimals
lat = '''7° 5' 16" S''''''
deg, minutes, seconds, direction =  re.split('[°\'"]', lat)
round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)'''

## Test
'''
#change coordinates to decimals
lon = '''97° 50' 2" W''''''
deg, minutes, seconds, direction =  re.split('[°\'"]', lon)
round((float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction.strip() in ['W', 'S'] else 1),6)
'''

SyntaxError: invalid syntax (4234492444.py, line 4)